In [ ]:
import qrcode
from qrcode.constants import ERROR_CORRECT_L

# URL to encode
url = "https://researcher.danhiggins.org/"

qr = qrcode.QRCode(
    version=1,  # controls size (1 to 40), 1 is 21x21 matrix
    error_correction=ERROR_CORRECT_L,
    box_size=10,  # size of each box in pixels
    border=4,     # thickness of border (boxes)
)

qr.add_data(url)
qr.make(fit=True)

img = qr.make_image(fill_color="black", back_color="white")
img.save("custom_qrcode.png")

In [1]:
import sys
sys.path.append('/Users/dan/Code/Python/pg_scaffold')
from app.core.db import SessionLocal
from app.crud import conference, attendee, poster, judge, judging_assignment, criterion, score, score_criterion
from app.schemas.conference import ConferenceCreate
from app.schemas.attendee import AttendeeCreate
from app.schemas.poster import PosterCreate
from app.schemas.judge import JudgeCreate
from app.schemas.criterion import CriterionCreate
from app.schemas.score import ScoreCreate
import datetime

In [2]:
db = SessionLocal()
skip = 0
limit = 100

# Read Tests     

In [3]:
from app.crud import conference as crud
db_objs = crud.conference.get_multi(db, skip=skip, limit=limit)
print(db_objs)


[<app.models.conference.Conference object at 0x10cc38a10>, <app.models.conference.Conference object at 0x10cc38a40>]


In [4]:
from app.schemas.conference import ConferenceRead  
# For Pydantic v2
pydantic_objs = [ConferenceRead.model_validate(o) for o in db_objs]

# For Pydantic v1:
# pydantic_objs = [ConferenceRead.from_orm(o) for o in db_objs]

for p in pydantic_objs:
    print(p.model_dump())  # or use p.dict()

ValidationError: 10 validation errors for ConferenceRead
attendees.0.conference
  Input should be a valid list [type=list_type, input_value=<app.models.conference.Co...e object at 0x10cc38a10>, input_type=Conference]
    For further information visit https://errors.pydantic.dev/2.11/v/list_type
attendees.1.conference
  Input should be a valid list [type=list_type, input_value=<app.models.conference.Co...e object at 0x10cc38a10>, input_type=Conference]
    For further information visit https://errors.pydantic.dev/2.11/v/list_type
attendees.2.conference
  Input should be a valid list [type=list_type, input_value=<app.models.conference.Co...e object at 0x10cc38a10>, input_type=Conference]
    For further information visit https://errors.pydantic.dev/2.11/v/list_type
attendees.3.conference
  Input should be a valid list [type=list_type, input_value=<app.models.conference.Co...e object at 0x10cc38a10>, input_type=Conference]
    For further information visit https://errors.pydantic.dev/2.11

In [ ]:
pydantic_objs = [ConferenceRead.attendees.model_validate(o) for o in db_objs]

## 1. Create Conference

In [ ]:
conference = ConferencesCreate(
    name="Comprehensive Test Conference", 
    location="Virtual", 
    start_date=datetime.date(2025, 9, 1), 
    end_date=datetime.date(2025, 9, 5)
)

db_conference = conferences.conferences.create(db, obj_in=conference)
print(f"Created conference: {db_conference.name}")

## 2. Create Attendees

In [ ]:
attendee1 = AttendeesCreate(
    first_name="Alice", 
    last_name="Smith", 
    email="alice.smith@example.com", 
    conference_id=db_conference.id
)
db_attendee1 = attendees.create(db, obj_in=attendee1)
print(f"Created attendee: {db_attendee1.first_name} {db_attendee1.last_name}")

attendee2 = AttendeesCreate(
    first_name="Bob", 
    last_name="Johnson", 
    email="bob.johnson@example.com", 
    conference_id=db_conference.id
)
db_attendee2 = attendees.create(db, obj_in=attendee2)
print(f"Created attendee: {db_attendee2.first_name} {db_attendee2.last_name}")

## 3. Create Poster

In [ ]:
poster = PostersCreate(
    board_location="B2", 
    poster_title="A Study of Test-Driven Development", 
    creator_id=db_attendee1.id
)
db_poster = posters.create(db, obj_in=poster)
print(f"Created poster: {db_poster.poster_title}")

## 4. Create Judge

In [ ]:
judge = JudgesCreate(person_id=db_attendee2.id, accepted=True)
db_judge = judges.create(db, obj_in=judge)
print(f"Created judge from attendee: {db_attendee2.first_name} {db_attendee2.last_name}")

## 5. Create Judging Assignment

In [ ]:
assignment = JudgingAssignmentsCreate(judge_id=db_judge.id, poster_id=db_poster.id)
db_assignment = judging_assignments.create(db, obj_in=assignment)
print(f"Created judging assignment for judge {db_judge.id} and poster {db_poster.id}")

## 6. Create Criteria

In [ ]:
criteria1 = CriteriaCreate(name="Clarity", max_score=10)
db_criteria1 = criteria.create(db, obj_in=criteria1)
print(f"Created criteria: {db_criteria1.name}")

criteria2 = CriteriaCreate(name="Methodology", max_score=15)
db_criteria2 = criteria.create(db, obj_in=criteria2)
print(f"Created criteria: {db_criteria2.name}")

## 7. Create Score

In [ ]:
score = ScoresCreate(assignment_id=db_assignment.id, comment="Good work!")
db_score = scores.create(db, obj_in=score)
print(f"Created score for assignment {db_assignment.id}")

## 8. Create Score Criteria

In [ ]:
score_crit1 = ScoreCriteriaCreate(score_id=db_score.id, criterion_id=db_criteria1.id, score=8)
db_score_crit1 = score_criteria.create(db, obj_in=score_crit1)
print(f"Created score criteria for score {db_score.id} and criteria {db_criteria1.name}")

score_crit2 = ScoreCriteriaCreate(score_id=db_score.id, criterion_id=db_criteria2.id, score=12)
db_score_crit2 = score_criteria.create(db, obj_in=score_crit2)
print(f"Created score criteria for score {db_score.id} and criteria {db_criteria2.name}")

## 9. Read, Update, and Delete

In [ ]:
# Read
print("--- Reading Data ---")
print(conferences.get(db, id=db_conference.id))
print(attendees.get(db, id=db_attendee1.id))
print(posters.get(db, id=db_poster.id))
print(judges.get(db, id=db_judge.id))
print(judging_assignments.get(db, id=db_assignment.id))
print(criteria.get(db, id=db_criteria1.id))
print(scores.get(db, id=db_score.id))
print(score_criteria.get(db, id=db_score_crit1.id))

# Update
print("--- Updating Data ---")
updated_poster = posters.update(db, db_obj=db_poster, obj_in={"poster_title": "An Updated Study of TDD"})
print(f"Updated poster title: {updated_poster.poster_title}")

# Delete
print("--- Deleting Data ---")
score_criteria.remove(db, id=db_score_crit1.id)
score_criteria.remove(db, id=db_score_crit2.id)
scores.remove(db, id=db_score.id)
criteria.remove(db, id=db_criteria1.id)
criteria.remove(db, id=db_criteria2.id)
judging_assignments.remove(db, id=db_assignment.id)
judges.remove(db, id=db_judge.id)
posters.remove(db, id=db_poster.id)
attendees.remove(db, id=db_attendee1.id)
attendees.remove(db, id=db_attendee2.id)
conferences.remove(db, id=db_conference.id)
print("Data deleted.")

# Verify Deletion
print("--- Verifying Deletion ---")
print(conferences.get(db, id=db_conference.id))
print(attendees.get(db, id=db_attendee1.id))
print(posters.get(db, id=db_poster.id))